In [144]:
import pandas as pd
import random as rnd 
import mysql.connector
from mysql.connector import Error
import plotly
import json
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pymysql
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
import networkx as nx
from networkx_viewer import Viewer
from functools import reduce
from scipy.spatial.distance import pdist, squareform
import jgraph
from cyjupyter import Cytoscape
from scipy.stats import kendalltau
from scipy.stats import weightedtau
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier

In [145]:
connection = mysql.connector.connect(host='127.0.0.1',
                                         database='employment',
                                         user='root',
                                         password='thanhn123')

In [146]:
def get_data(connection, descriptor_domain,filt = False,val = None):
    '''
    Return df in the SOCs-decriptor format of a specific domain 
    descriptor_domeain:
        work context
        knowledge
        skills
        gwas
        job_zones, filter
        Abilities, filter
    
    '''
    df= pd.DataFrame(data=None)
    if descriptor_domain == 'work_context':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_context
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'CX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'job_zones':
        if filt:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
            df = df[df['job_zone'].isin(val)]
        else:
            df= pd.read_sql('''
                SELECT title,job_zone FROM employment.job_zones
                LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
    elif descriptor_domain == 'abilities':
        if filt:
#             df= pd.read_sql('''
#             SELECT title, element_name, element_id,data_value FROM employment.''' + descriptor_domain + '''
#             LEFT JOIN employment.content_model_reference USING (element_id) 
#             LEFT JOIN employment.occupation_data USING (onetsoc_code); ''', con = connection)
#             df = df[df['element_id'].contains(val)]
            df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE element_id LIKE\''''+ val +'''%\'; ''', con = connection)
            
        else:
            df= pd.read_sql('''
            SELECT title, element_name,element_id, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_values':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_values
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'EX';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'interests':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.interests
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'OI';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'work_styles':
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.work_styles
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code)
            WHERE scale_id LIKE 'IM';''', con = connection)
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    elif descriptor_domain == 'name':
        df= pd.read_sql('''
            SELECT * FROM employment.occupation_data;''', con = connection)
    else:
        df= pd.read_sql('''
            SELECT title, element_name, data_value FROM employment.''' + descriptor_domain + '''
            LEFT JOIN employment.content_model_reference USING (element_id) 
            LEFT JOIN employment.occupation_data USING (onetsoc_code);''', con = connection)
        df = df.groupby(by=['title','element_name'])['data_value'].mean()
        df = df.reset_index().pivot(index = 'title', columns = 'element_name', values = 'data_value')
    return df

In [147]:
df_cps = pd.read_csv('../CPS Job Changes/JobChanges_2011to19.csv')

In [194]:
df_cps.sort_values(by = 'pct_tot', ascending = False)

,ONET18_SOC_LY,ONET18_SOC,ONET18_Title,ONET18_Title_LY,count,pct_tot,count_socly,pct_socly,in_grp,in_minor_grp,in_broad_occ,employment_projection_NewJob,wage_change_BetweenJobs
75823,41-1011,41-1012,First-Line Supervisors of Non-Retail Sales Wor...,First-Line Supervisors of Retail Sales Workers,734264.510000,4.395486e-01,2207170.940,33.267224,1,1,1,-8.3,85.257732
76141,41-1012,41-1011,First-Line Supervisors of Retail Sales Workers,First-Line Supervisors of Non-Retail Sales Wor...,603472.900000,3.612536e-01,999832.010,60.357429,1,1,1,-5.5,-46.021146
77141,41-2012,41-2031,Retail Salespersons,Gambling Change Persons and Booth Cashiers,509346.270000,3.049071e-01,2630467.290,19.363338,1,1,0,-0.6,-1.700405
76588,41-2011,41-2031,Retail Salespersons,Cashiers,509346.270000,3.049071e-01,2630467.290,19.363338,1,1,0,-0.6,6.772208
177,11-1021,11-1011,Chief Executives,General and Operations Managers,352402.430000,2.109567e-01,751730.590,46.878820,1,1,0,-10.0,83.034056
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88371,45-2092,25-1112,"Law Teachers, Postsecondary","Farmworkers and Laborers, Crop, Nursery, and G...",1.135947,6.800058e-07,184416.524,0.000616,0,0,0,6.9,346.293792
88369,45-2092,25-1082,"Library Science Teachers, Postsecondary","Farmworkers and Laborers, Crop, Nursery, and G...",1.135947,6.800058e-07,184416.524,0.000616,0,0,0,3.0,180.717341
88368,45-2092,25-1081,"Education Teachers, Postsecondary","Farmworkers and Laborers, Crop, Nursery, and G...",1.135947,6.800058e-07,184416.524,0.000616,0,0,0,4.8,157.524058
88367,45-2092,25-1072,"Nursing Instructors and Teachers, Postsecondary","Farmworkers and Laborers, Crop, Nursery, and G...",1.135947,6.800058e-07,184416.524,0.000616,0,0,0,17.6,193.257438


In [195]:
def distance_table(df, jz=False):
    
#     squareform(pdist(df.iloc[:, 1:])), columns=df.title.unique(), index=df.title.unique()
    dist = []
    name_dict = {}
    if jz:
        name_dict = df['title'].sort_values().reset_index()['title'].to_dict()
        dist = pdist(df[df.columns[1:]],'euclidean')
    else:
        for i in range(len(df.index)):
            name_dict[i] = df.index[i]
        dist = pdist(df,'euclidean') #Euclidean distance for 1XN
    df_dist = pd.DataFrame(squareform(dist)) #Euclidean distance for NxN
#     return df_dist
    columns = list(df_dist.index) 
    for i in range(len(df.index)): #convert 0 values to NaN
        df_dist.loc[i,i] = np.NaN
    mean = df_dist.mean().mean()
    std = df_dist.stack().std() 

    for col in columns: #Z-score calculated 
        df_dist[col] = (df_dist[col] - mean)/std
#     return df_dist
    return df_dist.rename(columns = name_dict, index = name_dict)

In [196]:
df_ab = get_data(connection,'abilities')
df_in = get_data(connection,'interests')
df_ws = get_data(connection,'work_styles')
df_wv = get_data(connection, 'work_values')
df_wc = get_data(connection,'work_context')
df_knowledge = get_data(connection,'knowledge')
df_gwas = get_data(connection,'work_activities')
df_jz = get_data(connection,'job_zones')
df_jz = df_jz[df_jz['title'] != 'Legislators']
df_skills = get_data(connection,'skills')



In [197]:
wc_dist = distance_table(df_wc)
knowledge_dist = distance_table(df_knowledge)
skills_dist = distance_table(df_skills)
gwas_dist = distance_table(df_gwas)
jz_dist = distance_table(df_jz,True)
wv_dist = distance_table(df_wv)
in_dist = distance_table(df_in)
ab_dist = distance_table(df_ab)
ws_dist = distance_table(df_ws)


In [198]:
to_drop = ['Fuel Cell Technicians', 'Green Marketers', 'Investment Underwriters',
       'Legislators', 'Methane/Landfill Gas Collection System Operators',
       'Methane/Landfill Gas Generation System Technicians']

In [199]:
# in_dist= in_dist.drop(index = to_drop)
# in_dist = in_dist.drop(columns = to_drop,axis = 1)
# wv_dist= wv_dist.drop(index = to_drop)
# wv_dist = wv_dist.drop(columns = to_drop,axis = 1)


In [200]:
def percentage_default(occ,df_cps,a,b):
    df_occ = df_cps[df_cps['ONET18_Title_LY'] == occ].sort_values(by = 'pct_socly', ascending = False)
    return df_occ[a:b]['pct_socly'].sum()

In [201]:
def percentage(occ,df_onet,df_cps,a,b):
    '''
    Get total percentage that ONET transition takes ups
    @param occ occupation of interests
    @param df_onet dataFrame for ONET
    @param df_cps dataFrame for CPS data
    @param a index of first position for onet
    @param index of last position
    
    return the percentage composition for the ath to bth place of ONET
    '''
    
    total = 0
    onet_list = df_onet.loc[occ].sort_values()[a:b].index.to_list()
    cps_occ = df_cps[df_cps['ONET18_Title_LY'] == occ]
    for occ in onet_list:
#         print(occ)
#         print(cps_occ[cps_occ['ONET18_Title'] == occ]['pct_tot'].values[0])
        val = cps_occ[cps_occ['ONET18_Title'] == occ]['pct_socly'].values
        if len(val) == 0:
            continue
        else:
            total += val[0]
    
    return total

In [202]:
def percentage_coverage(df_onet,df_cps):   
    cps_occ = set(df_cps['ONET18_Title_LY'].unique())
    onet_occ = set(df_onet.index.unique())
    avg_occ = 0
    count = 0
    relevant_occ = cps_occ & onet_occ
#     occ_dictionary = {}
    for occ in relevant_occ:
        cps_len = len(df_cps[df_cps['ONET18_Title_LY'] == occ]['ONET18_Title'])
        if cps_len > 20:
            n = percentage(occ,df_onet,df_cps,0,20)
            d = percentage_default(occ,df_cps,0,20)
            perc = n/d
            count += 1 
            avg_occ += perc
#             occ_dictionary[occ] = perc

    return avg_occ/count

In [203]:
X = np.array([skills_dist.to_numpy(), #skills 0
                       knowledge_dist.to_numpy(), #knowledge 1
                       in_dist.to_numpy(), #interest 2
                       gwas_dist.to_numpy(), #general work activities 3
                       wc_dist.to_numpy(), #work context 4
                       jz_dist.to_numpy(), #job zone 5
                       wv_dist.to_numpy(), #work value 6 
                       ab_dist.to_numpy(),],dtype=object) #abilites  8
#                        ws_dist.to_numpy()],dtype=object) #worksetting 8

In [204]:
mean_X = pd.concat([skills_dist, #skills 
                       knowledge_dist, #knowledge
                       in_dist, #interest
                       gwas_dist, #general work activities
                       wc_dist, #work context
                       jz_dist, #job zone
                       wv_dist, #work value
                       ab_dist,] #abilites
                     ).groupby(level =0).mean()

In [205]:
l = [skills_dist, #skills 
                       knowledge_dist, #knowledge
                       in_dist, #interest
                       gwas_dist, #general work activities
                       wc_dist, #work context
                       jz_dist, #job zone
                       wv_dist, #work value
                       ab_dist,]
test = np.array([0,1,1,1,1,1,1,1,])
# np.reshape()
# np.multiply(l,))
counter = 0
for a,b in zip(l,test):
    l[counter] = a*b
    counter += 1
    
l

[                                                    Accountants  Actors  \
 Accountants                                                 NaN    -0.0   
 Actors                                                     -0.0     NaN   
 Actuaries                                                  -0.0     0.0   
 Acupuncturists                                             -0.0    -0.0   
 Acute Care Nurses                                          -0.0     0.0   
 ...                                                         ...     ...   
 Wind Energy Project Managers                               -0.0     0.0   
 Wind Turbine Service Technicians                            0.0     0.0   
 Woodworking Machine Setters, Operators, and Ten...          0.0     0.0   
 Word Processors and Typists                                -0.0    -0.0   
 Zoologists and Wildlife Biologists                         -0.0     0.0   
 
                                                     Actuaries  Acupuncturists  \
 Acc

In [206]:
ws_dist.index.difference(skills_dist.index)

Index([], dtype='object')

In [207]:
X[0].shape

(968, 968)

In [208]:
# for i in range(9):
#     print(i)
#     print(X[i].shape)

In [209]:
arr = np.ones(shape = (8,968,968))

In [210]:
# y = np.dot(X,np.array([1,1,1,1,1,1,1,1,1,1,]))
# X * test
X.shape

(8,)

In [211]:
# X
# test =np.dot(X,arr)
# clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1)
# clf.fit(X, test)

In [212]:
# X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = [0,1,0,1]
# clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=100)
# clf.fit(X, y)
# clf.coef_
# y

In [238]:
onet_ = [skills_dist, #skills 
                       knowledge_dist, #knowledge
                       in_dist, #interest
                       gwas_dist, #general work activities
                       wc_dist, #work context
                       jz_dist, #job zone
                       wv_dist, #work value
                       ab_dist,
                       ws_dist]

In [239]:
def compute_loss(X,th):
    onet_list = X
    counter = 0
    for a,b in zip(X,th):
        onet_list[counter] = a*b
        counter += 1
    df_onet = pd.concat(onet_list).groupby(level=0).mean()
    p = percentage_coverage(df_onet,df_cps)
    return (p-1)**2

In [240]:
test = np.array([1,1,1,1,1,0,0,1,0])
compute_loss(onet_,test)

0.6253562920219617

In [250]:
def dJ(X,th):
    grad = [0]*9
    counter = 0
    for a,b in zip(X,th):
        grad[counter] = percentage_coverage(2*a*(a*b - 1),df_cps)
        print(grad[counter])
        counter += 1
    
    return grad
    

In [251]:
dJ(onet_,test)

0.007150046656931742
0.008696074544295794
0.011990032388265095
0.00995692181710918
0.008077736067252957
0.025172205776475182
0.02459780787874811
0.008422311272753726
0.02451871831744724


[0.007150046656931742,
 0.008696074544295794,
 0.011990032388265095,
 0.00995692181710918,
 0.008077736067252957,
 0.025172205776475182,
 0.02459780787874811,
 0.008422311272753726,
 0.02451871831744724]

In [132]:
def gradient_descent(J,dJ,max_iter,th):
    
    th = th
    th0 = th0
    step_size = 0.1
    for i in max_iter:
        dJ = 
        th = th - step_size*dJ

SyntaxError: invalid syntax (<ipython-input-132-277b8a4d5607>, line 1)

In [ ]:
graedient_descent(compute_loss,10,th0,th)